In [1]:
import tensorflow as tf
import pickle
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
import numpy as np
import os
import tqdm
import cv2
import matplotlib.pyplot as plt
from PIL import Image 
from tensorflow.keras import Sequential, Model
from tensorflow.python.client import device_lib
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3, ResNet50V2, Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img, img_to_array
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [18]:
train_dic = r'B:\proj\traffic-signs\data\train.p'
test_dic  = r'B:\proj\traffic-signs\data\test.p'
val_dic   = r'B:\proj\traffic-signs\data\valid.p'
test_files = r'C:\Users\nagim\Google Drive\projects\traffic_signs\traffic-signs-real\new\wildanimals31.jpg'
model_dir = r'C:\Users\nagim\Google Drive\projects\traffic_signs\model'
  
train = pickle.load(open(train_dic,'rb'))
test  = pickle.load(open(test_dic,'rb'))
val   = pickle.load(open(val_dic, 'rb' ))


x_train , y_train  = train['features'] , train['labels']
x_test  , y_test   = test['features']  , test['labels']
x_val   , y_val    = val['features']   , val['labels']



y_train = tf.one_hot(y_train,43)
y_test  = tf.one_hot(y_test,43)
y_val   = tf.one_hot(y_val,43) 

In [9]:
def create_model() :
    model=Sequential()

    model.add(Conv2D(filters = 256,  kernel_size = (5,5), padding='same', activation='relu', input_shape = (32, 32, 3)))
    model.add(MaxPool2D(pool_size=(2, 2), padding = 'same'))

    model.add(Conv2D(filters = 512, kernel_size = (5,5), padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), padding = 'same'))

    model.add(Conv2D(filters = 1024, kernel_size = (5,5), padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), padding = 'same'))


    model.add(Flatten())

    model.add(Dense(units= 1024, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(units= 512, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(units= 256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(units= 100, activation='relu'))

    model.add(Dense(units= 43, activation='softmax'))

    model.compile(optimizer= Adam(), loss= CategoricaCrossentropy() ,metrics='accuracy')

In [21]:
if os.path.exists(model_dir + '\saved_model.pb'):
    model = load_model(model_dir)
    print('loadded')
else:
    model = create_model

loadded


In [11]:
model.fit(x=x_train, y=y_train, batch_size=512,epochs=50,validation_data=(x_val,y_val))

Epoch 1/50
68/68 [==============================] - 31s 459ms/step - loss: 10.6075 - accuracy: 0.0535 - val_loss: 3.4371 - val_accuracy: 0.1195
Epoch 2/50
68/68 [==============================] - 27s 394ms/step - loss: 3.0887 - accuracy: 0.1350 - val_loss: 2.6178 - val_accuracy: 0.2120
Epoch 3/50
68/68 [==============================] - 27s 399ms/step - loss: 2.3131 - accuracy: 0.2997 - val_loss: 1.7367 - val_accuracy: 0.4565
Epoch 4/50
68/68 [==============================] - 27s 401ms/step - loss: 1.4823 - accuracy: 0.5456 - val_loss: 1.1797 - val_accuracy: 0.6728
Epoch 5/50
68/68 [==============================] - 27s 404ms/step - loss: 0.9024 - accuracy: 0.7255 - val_loss: 0.7275 - val_accuracy: 0.8107
Epoch 6/50
68/68 [==============================] - 28s 406ms/step - loss: 0.5717 - accuracy: 0.8401 - val_loss: 0.4478 - val_accuracy: 0.8848
Epoch 7/50
68/68 [==============================] - 28s 406ms/step - loss: 0.3156 - accuracy: 0.9105 - val_loss: 0.3984 - val_accuracy: 0.898

In [22]:
model.evaluate(x= x_test , y= y_test, batch_size= 512 , verbose=1)

25/25 [==============================] - 1s 45ms/step - loss: 0.3967 - accuracy: 0.9511


[0.3967425227165222, 0.9510688781738281]

In [23]:
x = load_img(test_files)
x = img_to_array(x)
x = np.expand_dims(x, axis=0)
y = model.predict(x)
y.argmax()

31

In [7]:
model = load_model(model_dir)

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 256)       19456     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 512)       3277312   
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 1024)        13108224  
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 1024)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 2048)       

In [40]:
model.save(model_dir)

INFO:tensorflow:Assets written to: C:\Users\nagim\Google Drive\projects\traffic_signs\model\assets
